In [ ]:
import os
import json
import numpy as np
from tqdm.auto import tqdm
import math

In [ ]:
from adapted import AdaptedCameras

In [ ]:
# constants
REQUESTED_ROOMS = [ 'Bedroom', 'MasterBedroom', 'SecondBedroom']

In [ ]:
# folder and path
model_info_json_path = "/home/ubuntu/research/3D-FRONT/3D-FUTURE-model/model_info.json"
scene_folder = "/home/ubuntu/research/3D-FRONT/3D-FRONT/"

In [ ]:
# load model category
model_category_info = json.load(open(model_info_json_path, "r"))
model_id2category = {item['model_id']:item['category'] for item in model_category_info}

In [ ]:
scene_name_list = os.listdir(scene_folder)

In [ ]:
all_room_info = {}
for idx in tqdm(range(0, 5088)): #len(scene_name_list)
    scene_file_path = scene_name_list[idx]

    camera = AdaptedCameras(os.path.join(scene_folder, scene_file_path))

    room_dict = camera.scene_content.dict_room

    for iname in room_dict.keys():
        room = room_dict[iname]
        if room.type not in REQUESTED_ROOMS:
            continue
        if iname not in camera.floor_info.keys():
            continue

        
        # get room floor infomation
        floor = camera.floor_info[iname]['floor']
        floor_x = [item[0] for item in floor]
        min_x = np.min(floor_x)
        max_x = np.max(floor_x)
        
        floor_z = [item[1] for item in floor]
        min_z = np.min(floor_z)
        max_z = np.max(floor_z)
        
        # get room bbox
        room_box = [max_x - min_x, 2.5, max_z - min_z] # max: x, y, z
        

        room_dict43DSLN = {'id':iname, 'valid_objects':[], 'bbox':room_box}
        # furniture
        furnitures = room.children_for_furniture
        for ifurniture in furnitures:
            uid = ifurniture['id']
            if uid not in camera.model_dict.keys():
                continue

            model_info = camera.model_dict[uid]
            cate_id = model_info['category_id']

            #if cate_id not in model_id2category:
            #    continue

            cate = model_id2category[model_info['jid']]

            furni_dict = {
                'jid': model_info['jid'],
                'category_id': model_info['category_id'],
                'size': model_info['box'],
                'scale': list(ifurniture['scale']),
                'pos': list(ifurniture['pos']),
                'rot': list(ifurniture['rot']),
                'category':cate,
                'floor': floor
            }
            
            # calculate bbox and normalize
            bbox = [[0,0,0],[0,0,0]]
            bbox[0][0] = furni_dict['pos'][0] - furni_dict['size']['xLen'] / 200.0
            bbox[0][1] = furni_dict['pos'][1]
            bbox[0][2] = furni_dict['pos'][2] - furni_dict['size']['zLen'] / 200.0
            bbox[1][0] = furni_dict['pos'][0] + furni_dict['size']['xLen'] / 200.0
            bbox[1][1] = furni_dict['pos'][1] + furni_dict['size']['yLen'] / 200.0
            bbox[1][2] = furni_dict['pos'][2] + furni_dict['size']['zLen'] / 200.0
            
            furni_dict['unormalized_bbox'] = [[v for v in u] for u in bbox]
            
            bbox[0][0] =  (bbox[0][0] - min_x)/(max_x - min_x)
            bbox[0][1] =  bbox[0][1] / 2.5
            bbox[0][2] =  (bbox[0][2] - min_z)/(max_z - min_z)
            bbox[1][0] =  (bbox[1][0] - min_x)/(max_x - min_x)
            bbox[1][1] =  bbox[1][1] / 2.5 # floor height = 2.5
            bbox[1][2] =  (bbox[1][2] - min_z)/(max_z - min_z)
            
            
            furni_dict['new_bbox'] = bbox
            
            
            # get type
            furni_dict['type'] = cate.split(" ")[-1].lower() 
            
            # get rotation
            
            t2 = +2.0 * (furni_dict['rot'][3] * furni_dict['rot'][1] - furni_dict['rot'][2] * furni_dict['rot'][0])
            t2 = +1.0 if t2 > +1.0 else t2
            t2 = -1.0 if t2 < -1.0 else t2
            pitch_y = math.asin(t2) / math.pi * 180
            
            furni_dict['rotation'] = pitch_y // 15 % 24
            
            
            room_dict43DSLN['valid_objects'].append(furni_dict)
        
        if len(room_dict43DSLN['valid_objects']) > 0:
            all_room_info[idx] = room_dict43DSLN
        #break #debug

In [ ]:
camera.scene_content

In [ ]:
all_category_list = []
for oom in all_room_info.values():
    for bject in oom['valid_objects']:
        category = bject['category']
        if category not in all_category_list:
            all_category_list.append(category)

In [ ]:
all_category_list = sorted(all_category_list)

In [ ]:
all_category_list

In [ ]:
all_category_list_cleaned = [item.split(" ")[-1].lower() for item in all_category_list]

In [ ]:
valid_types = list(set(all_category_list_cleaned))

In [ ]:
json.dump(valid_types, open("metadata43DSLN/valid_types_a.json", "w"))

In [ ]:
json.dump(all_room_info, open("metadata43DSLN/all_room_info_a.json", "w"))

In [ ]:
len(all_room_info)